In [12]:
import pandas as pd
import numpy as np
import os
import datetime
month = "202212"
weekdays=21
specials=10

in_file = os.path.join(os.getcwd(), "raw_data", month, "origin_destination_train_" + month + ".csv")

df = pd.read_csv(in_file)

In [13]:
same_zones = [
    "NS1", "EW24", "NS25", "EW13", "NS26", "EW14", "CC4", "DT15"
]
#similarly, sanity checks for JUR, SKG, PGL, ORC

same_zones_div_2 = [
    "EW24/NS1", "NE16/STC", "NE17/PTC"
]

# unpaid interchanges. we don't want to sum them since we can't tell who's changing and who's exiting.
unpaid_ints = [
    "BP6/DT1", "PB6/DT1", "EW2/DT32", "DT32/EW2", "NS21/DT11", "DT11/NS21"
]

# special handing for 202212 onwards - interchange stations were not summed
interchanges = {
    "CC10/DT26": ["CC10", "DT26"] ,
    "CC17/TE9": ["CC17", "TE9"] ,
    "CC19/DT9": ["CC19", "DT9"] ,
    "CC4/DT15": ["CC4", "DT15"] ,
    "CE1/DT16": ["CE1", "DT16"] ,
    "CG1/DT35": ["CG1", "DT35"] ,
    "DT10/TE11": ["DT10", "TE11"],
    "EW12/DT14": ["EW12", "DT14"] ,
    #"EW14/NS26": ["EW14", "NS26"] ,
    "EW16/NE3/TE17": ["EW16", "NE3", "TE17"],
    #"EW2/DT32": ["EW2", "DT32"] ,
    "EW21/CC22": ["EW21", "CC22"] ,
    "EW24/NS1": ["EW24", "NS1"] ,
    "EW8/CC9": ["EW8", "CC9"] ,
    "NE1/CC29": ["NE1", "CC29"] ,
    "NE12/CC13": ["NE12", "CC13"] ,
    "NE16/STC": ["NE16", "STC"] ,
    "NE17/PTC": ["NE17", "PTC"] ,
    "NE4/DT19": ["NE4", "DT19"] ,
    "NE7/DT12": ["NE7", "DT12"] ,
    "NS17/CC15": ["NS17", "CC15"] ,
    #"NS21/DT11": ["NS21", "DT11"] ,
    "NS24/NE6/CC1": ["NS24", "NE6", "CC1"],
    #"NS25/EW13": ["NS25", "EW13"] ,
    "NS27/CE2/TE20": ["NS27", "CE2", "TE20"],
    "NS4/BP1": ["NS4", "BP1"] ,
    "NS9/TE2": ["NS9", "TE2"] ,
    #"PB6/DT1": ["PB6", "DT1"] ,
    "TE14/NS22": ["TE14", "NS22"] ,

}

In [14]:
df[df["ORIGIN_PT_CODE"] == "TE7"].to_csv("te7_orig.csv")

In [10]:
df_tel = df[df['ORIGIN_PT_CODE'].isin(sum(interchanges.values(), []))]
for code, code_array in interchanges.items():
    df_telint = df[df['ORIGIN_PT_CODE'].isin(code_array)]
    column_map = {col: "first" for col in df.columns}
    column_map["TOTAL_TRIPS"] = "sum"
    df_telint2 = df_telint.groupby(["DAY_TYPE", "TIME_PER_HOUR", "DESTINATION_PT_CODE"], as_index=False).aggregate(column_map)
    df_telint2["ORIGIN_PT_CODE"] = code
    df = df[~df["ORIGIN_PT_CODE"].isin(code_array)]
    df_telint2.reset_index(drop=True, inplace=True)
    df = pd.concat([df, df_telint2])

df_tel = df[df['DESTINATION_PT_CODE'].isin(sum(interchanges.values(), []))]
for code, code_array in interchanges.items():
    df_telint = df[df['DESTINATION_PT_CODE'].isin(code_array)]
    column_map = {col: "first" for col in df.columns}
    column_map["TOTAL_TRIPS"] = "sum"
    df_telint2 = df_telint.groupby(["DAY_TYPE", "TIME_PER_HOUR", "ORIGIN_PT_CODE"], as_index=False).aggregate(column_map)
    df_telint2["DESTINATION_PT_CODE"] = code
    df = df[~df["DESTINATION_PT_CODE"].isin(code_array)]
    df_telint2.reset_index(drop=True, inplace=True)
    df = pd.concat([df, df_telint2])

df['multiplier'] = df['DAY_TYPE']

# not needed wef 202212
#df = df.replace({'ORIGIN_PT_CODE': interchange_codes, 'DESTINATION_PT_CODE': interchange_codes, 'multiplier': {'WEEKENDS/HOLIDAY': specials, 'WEEKDAY': weekdays}})
df = df.replace({'multiplier': {'WEEKENDS/HOLIDAY': specials, 'WEEKDAY': weekdays}})

df = df[~df['ORIGIN_PT_CODE'].isin(same_zones)]
df = df[~df['ORIGIN_PT_CODE'].isin(unpaid_ints)]

df = df[~df['DESTINATION_PT_CODE'].isin(same_zones)]
df = df[~df['DESTINATION_PT_CODE'].isin(unpaid_ints)]

df_filter = df[df['ORIGIN_PT_CODE'].isin(same_zones_div_2)]
df = df[~df['ORIGIN_PT_CODE'].isin(same_zones_div_2)]
df_filter['TOTAL_TRIPS'] = (df_filter['TOTAL_TRIPS'] / 2)
df = pd.concat([df, df_filter])

df_filter = df[df['DESTINATION_PT_CODE'].isin(same_zones_div_2)]
df = df[~df['DESTINATION_PT_CODE'].isin(same_zones_div_2)]
df_filter['TOTAL_TRIPS'] = (df_filter['TOTAL_TRIPS'] / 2)
df = pd.concat([df, df_filter])

In [11]:
df[df["ORIGIN_PT_CODE"] == "TE7"].to_csv("te7.csv")